In [ ]:
import pandas as pd
import numpy as np
import json



#cargo los datasets
#si agrego  dtype='unicode' evito algunos errores sobre tipos de datos
dfMovies = pd.read_csv(r'datasets\movies_dataset.csv', dtype='unicode')
dfCredits= pd.read_csv(r'datasets\credits.csv', dtype='unicode')

pd.set_option("display.max_rows", 2000, "display.max_columns", None)

In [ ]:
#TRANSFORMACIONES VARIAS ---> NO OLVIDAR CORRER LO QUE ESTA COMENTADO

#lleno NaNs con 0s en las columnas revenue y budget
#elimino rows que contienen un NaN en la columna release_date
#elimino columnas que no se utilizaran
try:
    dfMovies['revenue'] = dfMovies['revenue'].fillna(0)
    dfMovies['budget'] = dfMovies['budget'].fillna(0)
    dfMovies = dfMovies.dropna(subset='release_date')
    dfMovies= dfMovies.drop(columns=["video","adult","imdb_id" ,"original_title", "poster_path", "homepage"])
except:
    print("estas transformaciones ya se ejecutaron")


In [ ]:
#FUNCIONES

#desanida columna y devuelve un dataframe
def desanidarColumna(columna):
    lista = []
    listaDict=[]
    counter=0
    for k, v in columna.items():
        if isinstance(v, str):
            lista.append(v)
        else:
            lista.append(None)
    for i in lista:   
                try:
                    i=json.loads(lista[counter].replace("'", '"'))
                    listaDict.append(i)
                    counter+=1
                except:
                    listaDict.append(None)
                    counter+=1
    listaDict=pd.json_normalize(listaDict)
    return listaDict

def desanidarVariasColumnas(columna):
    lista=[]
    for col in columna:            
        #normalizo cada columna
        colAnidada=pd.json_normalize(columna[col])
        lista.append(colAnidada) #meto cada columna en una lista

        #armo un df concatenando cada columna de la lista
        nuevoDF=pd.concat(lista)
    return nuevoDF

#esta funcion crea nuevos dfs para usarlos en las funcion de fastaPI segun las columnas que necesito
def creaDfFuncionesFastAPI(columnas):
    nuevoDF = []
    for i in dfMovies["movie_title"]:
        row=dfMovies.loc[dfMovies.movie_title == i, columnas]
        nuevoDF.append(row)
    #usando concatenar si ambos dfs estan en una lista, se unen facilmente
    nuevoDF = pd.concat(nuevoDF, ignore_index=True)
    nuevoDF = nuevoDF.dropna()
    return nuevoDF

In [ ]:
#desanido belongs to collection
franquicia = (desanidarColumna(dfMovies['belongs_to_collection']))

#inserto nueva columna y elimino la vieja
dfMovies= dfMovies.drop(columns=["belongs_to_collection"])
dfMovies.insert(1, "franquicia", franquicia["name"])

In [ ]:
#la columna generos hay que desanidarle cada columna despues de aplciarle la funcion

dfGeneros = (desanidarColumna(dfMovies['genres'])) #le aplico la primera funcion
dfGeneros=desanidarVariasColumnas(dfGeneros) #desanido cada columna con la segunda funcion      
dfGeneros=dfGeneros.iloc[:45378] #esto es cualquier cosa pero bueno, funciona xd
                                 #uso iloc para obtener solo hasta el row 45378 porque sino tengo errores
                                 
#inserto nueva columna y elimino la vieja
dfMovies.insert(1, "movie_genre", dfGeneros["name"])
dfMovies= dfMovies.drop(columns=["genres"])

In [ ]:
#desanido tabla production_companies
dfCompanies=desanidarColumna(dfMovies['production_companies'])
dfCompanies=desanidarVariasColumnas(dfCompanies)
dfCompanies= dfCompanies.iloc[:45379]

#inserto nueva columna y elimino la vieja
dfMovies.insert(7, "production_company", dfCompanies["name"])
dfMovies= dfMovies.drop(columns=["production_companies"])

In [ ]:
#desanido production_countries
dfProdCountries=desanidarVariasColumnas(desanidarColumna(dfMovies["production_countries"]))
dfProdCountries = dfProdCountries.iloc[:45379]

#inserto nueva columna y elimino la vieja
dfMovies.insert(8, "production_country", dfProdCountries["name"])
dfMovies= dfMovies.drop(columns=["production_countries"])

In [ ]:
#desanido spoken_languages
dfSpokenLan=desanidarVariasColumnas(desanidarColumna(dfMovies["spoken_languages"]))
dfSpokenLan = dfSpokenLan.iloc[:45379]

#inserto nueva columna y elimino la vieja
dfMovies.insert(12, "language", dfSpokenLan["name"]) #puedo elegir entre poner la abreviacion o la palabra entera
dfMovies= dfMovies.drop(columns=["spoken_languages"])

In [ ]:
#tablas que contienen fechas:
    #release_date
release_year = dfMovies["release_date"]
#cambio valores de fechas invalidas a fechas default
release_year[19730] = "1900-01-01"
release_year[29503] = "1900-01-01"
release_year[35587] = "1900-01-01"

#obtengo solo año de cada string de fecha y lo meto en release_year
release_year = pd.to_datetime(release_year).dt.year

#inserto nueva columan release_year
dfMovies.insert(10, "release_year", release_year)

#se supone que transforme la columna en date type pero es un int REVISAR

In [ ]:
#comienzo a armar columna return

budget=dfMovies["budget"] #debo reemplazar esos valores porque sino me da error
budget[19730] = 0
budget[29503] = 0
budget[35587] = 0

#paso a float
dfMovies["revenue"]=pd.DataFrame(dfMovies["revenue"]).astype(float)
dfMovies["budget"]=pd.DataFrame(dfMovies["budget"]).astype(float)

#creo la columna return y relleno con 0 cuando el resultado es None
dfMovies["return"]=round(dfMovies["revenue"]/dfMovies["budget"], 2)
dfMovies['return'] = dfMovies['return'].fillna(0)

#reemplazo valores infinitos
dfMovies.replace([np.inf, -np.inf], 0, inplace=True)

#para revisar si hay valores infinitos
counter=0
for i in dfMovies["return"]:
    if i == float("inf"):
        print(i)
        counter+=1
print(counter)


In [ ]:
#desanido la columna crew y la dejo en un df separado (tarda como 13 minutos en armarse)

dfCrew = pd.DataFrame()
for i in range(len(dfCredits)):
    stringsCrew = dfCredits.crew[i]
    dictionaryCrew = eval(stringsCrew) #esto es para pasarlo de string a lista y poder armar el df
    dictionaryCrew = pd.DataFrame(dictionaryCrew)
    dictionaryCrew["id"] = dfCredits.id[i]  #reemplazo el id del df con el de dfCredits para poder unirlo a su pelicula
    dfCrew = pd.concat([dfCrew, dictionaryCrew])
    #dfCrew = dfCrew.reset_index(drop=True) #si descomento el reindex tarda mas del doble

In [ ]:
#desanido la columna cast y la dejo en un df separado (tarda como 17 minutos en armarse)
#dfCast = pd.DataFrame()
#for i in range(len(dfCredits)):
#    stringsCast = dfCredits.cast[i]
#    dictionaryCast = eval(stringsCast) #esto es para pasarlo de string a lista y poder armar el df
#    dictionaryCast = pd.DataFrame(dictionaryCast)
#    dictionaryCast["id"] = dfCredits.id[i] #reemplazo el id del df con el de dfCredits para poder unirlo a su pelicula
#    dfCast = pd.concat([dfCast, dictionaryCast])
    #dfCast = dfCast.reset_index(drop=True) #si descomento el reindex tarda mas del doble

In [ ]:
#cambio de posicion la columna "title" y le cambio el nombre a "movie_title"
titles=pd.DataFrame(dfMovies["title"])
dfMovies.insert(0,"movie_title" ,titles)
dfMovies=dfMovies.drop(columns=["title"])

#cambio nans en todas las columnas por "Unkown Value"

#columns = [a for a in dfMovies.columns]
#for i in range(len(columns)):
#    dfMovies[columns[i]]=dfMovies[columns[i]].fillna("Unkown Value")

#valores raros en id
id=dfMovies["id"]

id[19730] = 1111111
id[29503] = 2222222
id[35587] = 3333333


In [ ]:
#para exportar csv e ir probando cargarlo a la API
#dfMovies.to_csv(r"C:\Users\\Administrator\\Desktop\\dfMovies.csv")

In [ ]:
#dfMovies["release_date"]
                                                       #%d&%m&%y
#dfMovies["release_date"] = pd.to_datetime(dfMovies["release_date"], format="%Y-%m-%d")
#month = dfMovies["release_date"].dt.month_name()
#dfMovies[""]


#"710": "710", REVISAR ESTE ID, INDEX NO SE. A PARTIR DE ACA YA NO COINCIDEN EL ID O ALGO ASI.

In [ ]:
#PRIMERA FUNCION DATAFRAME dfPeliculasIdiomas

#dataset funcion peliculas idioma
#dfMoviesDuration=creaDfFuncionesFastAPI(["movie_title", "language", "original_language"])

#guardo dataset para la primera funcion
#dfPeliculasIdiomas.to_csv(r"C:\Users\\Administrator\\Desktop\\dfPeliculasIdiomas.csv")

In [ ]:
#SEGUNDA FUNCION DATAFRAME dfMoviesDuration

#nuevo df con la info que necesito para la segunda funcion
#dfMoviesDuration=creaDfFuncionesFastAPI(["movie_title", "release_year", "runtime"])

#guardo dataset para la segunda funcion
#dfMoviesDuration.to_csv(r"C:\Users\\Administrator\\Desktop\\dfMoviesDuration.csv")

In [ ]:
#TERCERA FUNCION DATAFRAME dfFranquicia

#POR EL MOMENTO, GANANCIA TOTAL VA A SER GIUAL A REVENUE
#dfMovies["ganancia_total"]=round(dfMovies["revenue"]-dfMovies["budget"], 2)

pd.options.display.float_format = '{:.2f}'.format #porque sino me tira numeros con notacion cientifica

#solo obtengo 5381 registros ya que hay muchos campos nulos tanto en budget como en revenue
#dfMovies.loc[(dfMovies["revenue"] > 0) & (dfMovies["budget"] > 0) , ["movie_title","franquicia","budget","revenue", "ganancia_total"]]

#la funcion toma como parametro FRANQUICIA
#necesito CANTIDAD de peliculas de esa franquicia
#GANACIA total (revenue)
#PROMEDIO dela ganancia total

#filtro revenue mayor a 0 y me quedo con columnas que necesito
dfFiltrado = dfMovies.loc[(dfMovies["revenue"] > 0), ["franquicia","movie_title","revenue"]]

#hago group by por franquicia y agrego cada una de las maneras para agrupar las demas columnas
dfFranquicia = dfFiltrado.groupby("franquicia").agg(
    movie_count=("movie_title", "count"),
    ganancia_total=("revenue", "sum"),
    ganancia_promedio=("revenue", "mean")).reset_index()

#guardo dataset para la tercera funcion
dfFranquicia.to_csv(r"C:\Users\\Administrator\\Desktop\\dfFranquicia.csv")




In [ ]:
#CUARTA FUNCION DATAFRAME dfPaises

#def peliculas_pais( Pais: str ): Se ingresa un país (como están escritos en el dataset, no hay que traducirlos!),
# retornando la cantidad de peliculas producidas en el mismo.
#    Ejemplo de retorno: Se produjeron X películas en el país X

#creo df para la cuarta funcion
#dfPaises = creaDfFuncionesFastAPI(["production_country", "movie_title"])

#agrupo por pais y cuento las peliculas
#dfPaises = dfPaises.groupby("production_country").agg(
#    movie_count=("movie_title", "count")).reset_index()

#exporto df para cuarta funcion
#dfPaises.to_csv(r"C:\Users\\Administrator\\Desktop\\dfPaises.csv")


In [ ]:
#QUINTA FUNCION DATAFRAME dfProductora

#def productoras_exitosas( Productora: str ): Se ingresa la productora, entregandote el revunue total y la
# cantidad de peliculas que realizo.
#Ejemplo de retorno: La productora X ha tenido un revenue de x

#dfProductora = creaDfFuncionesFastAPI(["production_company", "movie_title","revenue"])

dfProductora = dfMovies.loc[(dfMovies["revenue"] > 0), ["production_company","movie_title","revenue"]]

dfProductora = dfProductora.groupby("production_company").agg(
    movie_count =("movie_title", "count"),
    ganancia_total = ("revenue", "sum")
).reset_index()

#exporto df para quinta funcion
dfProductora.to_csv(r"C:\Users\\Administrator\\Desktop\\dfProductora.csv")

In [ ]:
#armo dfs que voy a usar para la sexta funcion

#dfMovieReturn = creaDfFuncionesFastAPI(["id", "return"])

#dfMovieInfo = creaDfFuncionesFastAPI(["id", "movie_title","release_date", "budget","revenue","return"])

#dfDirectores = dfCrew.loc[(dfCrew["job"] == "Director"), ["id","job","name"]]

In [185]:
#SEXTA FUNCION DATAFRAME 1 dfDirectoresExito

#filtro para no traer los returns en 0
dfMovieReturn = dfMovieReturn.loc[dfMovieReturn["return"]>0]

#para revisar cuantos directores hay por pelicula
dfDirectoresPorPelicula = dfDirectores.groupby("id").agg(
    job_count=("job", "count"),
    name_count=("name", "count")
).reset_index()
dfDirectoresPorPelicula[dfDirectoresPorPelicula["job_count"]>1]

#hago join para poder calcular el return de cada director
dfDirectoresExito = dfDirectores.merge(dfMovieReturn, on="id",how="left")

#dropeo columnas que no me sirven
dfDirectoresExito=dfDirectoresExito.drop(columns=["job", "id"])

#hago group by director name
dfDirectoresExito = dfDirectoresExito.groupby("name").agg(
    director_return = ("return", "sum")
)

#resultado sin tener en cuenta cuando no hay return diponible para el director
dfDirectoresExito = dfDirectoresExito.loc[dfDirectoresExito["director_return"]>0]

#exporto uno de los dfs que voy a usar para la sexta funcion
dfDirectoresExito.to_csv(r"C:\Users\\Administrator\\Desktop\\dfDirectoresExito.csv")

In [183]:
#SEXTA FUNCION DATAFRAME 2 dfDirectoresInfoPeliculas

#filtro para no traer valores en 0
dfMovieInfo = dfMovieInfo.loc[dfMovieInfo["return"]>0]
#hago join para poder calcular el return de cada director
dfDirectoresInfoPeliculas = dfMovieInfo.merge(dfDirectores, on="id",how="left")

#hago reindex
dfDirectoresInfoPeliculas = dfDirectoresInfoPeliculas.reindex(columns=["id","name", "job","movie_title", "release_date",
                                                                "budget", "revenue", "return"])
#dropeo columnas que no me sirven
dfDirectoresInfoPeliculas=dfDirectoresInfoPeliculas.drop(columns=["job", "id"])                                                                 

#exporto el otro de los dfs que voy a usar para la sexta funcion
dfDirectoresInfoPeliculas.to_csv(r"C:\Users\\Administrator\\Desktop\\dfDirectoresInfoPeliculas.csv")        